In [3]:
from platform import python_version

print(python_version())

3.7.3


In [2]:
# import libraries 
import itertools
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM


ImportError: DLL load failed: The specified procedure could not be found.

# Initialisation

In [ ]:
# eeg1 and events1 are the test data from a single person
# code assumes eeg1 and events1 are csv files in the current working directory

# eeg1 = pd.read_csv("eeg1.csv", delimiter="\t")
# new_columns = eeg1.columns.values 
# new_columns[0] = 'time'     
# new_columns[33] = 'sample' 
# eeg1.columns = new_columns

# events1 = pd.read_csv("events1.csv") #, delimiter="\t"

In [ ]:
# full dataset parameters

# define model parameters
#samples =   # how many trials of eeg data
n_features = 66  # how many channels of eeg in each sample
time_steps = 750 # how many ms was each sample run for
event_types = 54 #len(set(y))  # how many different event types (light, sound, etc) are there # 6 large, 4 smol

In [ ]:
def one_hot_events(events):
    # helper function for one-hot encoding the events
    events_list = list(events)
    lb = preprocessing.LabelBinarizer()
    lb.fit(events_list)
    events_1hot = lb.transform(events_list)
    return events_1hot, lb

def invert_one_hot(events, lb):
    # function for decoding one-hot, binarizer made in one_hot_events
    inv_events = lb.inverse_transform(events)
    return inv_events

In [ ]:
# y = one_hot_events(y)

In [ ]:
def split(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)
    return X_train, X_val, X_test, y_train, y_val, y_test

# TRAIN

According to **Generative Adversarial Networks Conditioned by Brain Signals**, feature encoder/classifier is a LSTM RNN. It consists of a standard LSTM layers followed by a nonlinear layer. At each time step, input s(; t) (i.e., the set of values from all channels at time t) is fed into the LSTM layer; when all time steps have been processed, the final output state of the LSTM goes into a fully-connected layer with ReLU non-linearity.



- LSTM -> fully connected RELU -> softmax classification

- Optimiser: Adam, initial learning rate = 0.001

- Minibatches of size 16

- Layer sizes: 128

In [ ]:


# code for building an LSTM with 100 neurons and dropout. Runs for 50 epochs

model = Sequential()
model.add(LSTM(128, return_sequences=False, input_shape=(time_steps, n_features)))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(54, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
# based on papers, should be adam & learning rate 0.001. But this worked for some guys

model.fit(X_train, y_train, batch_size=16, epochs=50)
score = model.evaluate(X_test, y_test, batch_size=16)

In [ ]:
print(score)

In [ ]:
print("Accuracy: {:.2%}".format(score[1]))

## for playing

In [ ]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit(["a","b","c","d","e","e"])
print(lb.classes_)


In [ ]:
x = lb.transform(["a","b","c","d","e","e"])
print(x)


In [ ]:
print(lb.inverse_transform(x))